In [1]:
from __future__ import division
from datetime import datetime, timedelta,date
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns

import chart_studio.plotly as py
import plotly.offline as pyoff
import plotly.graph_objs as go
pyoff.init_notebook_mode()

import warnings
warnings.filterwarnings("ignore")


In [2]:
df=pd.read_csv("../datasets/sales_prediction_train.csv")

In [3]:
df.tail()

date  store  item  sales
912995  2017-12-27     10    50     63
912996  2017-12-28     10    50     59
912997  2017-12-29     10    50     74
912998  2017-12-30     10    50     62
912999  2017-12-31     10    50     82

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 913000 entries, 0 to 912999
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   date    913000 non-null  object
 1   store   913000 non-null  int64 
 2   item    913000 non-null  int64 
 3   sales   913000 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 27.9+ MB


## Data Preprocessing

In [5]:
df['date']=pd.to_datetime(df['date'])

In [6]:
df['date'] = df['date'].dt.year.astype('str') + '-' + df['date'].dt.month.astype('str') + '-01'
df['date'] = pd.to_datetime(df['date'])

In [7]:
df = df.groupby('date').sales.sum().reset_index()

In [8]:
df.head()

date   sales
0 2013-01-01  454904
1 2013-02-01  459417
2 2013-03-01  617382
3 2013-04-01  682274
4 2013-05-01  763242

## Data Transformation

In [9]:
#plot monthly sales
plot_data = [
    go.Scatter(
        x=df['date'],
        y=df['sales'],
    )
]
plot_layout = go.Layout(
        title='Montly Sales'
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
pyoff.iplot(fig)

In [10]:
#creating a new dataframe to model the difference
df_diff=df.copy()
df_diff['prev_sales']=df_diff['sales'].shift(1)

In [11]:
df_diff = df_diff.dropna()
df_diff['diff'] = (df_diff['sales'] - df_diff['prev_sales'])

In [12]:
df_diff.head()

date   sales  prev_sales      diff
1 2013-02-01  459417    454904.0    4513.0
2 2013-03-01  617382    459417.0  157965.0
3 2013-04-01  682274    617382.0   64892.0
4 2013-05-01  763242    682274.0   80968.0
5 2013-06-01  795597    763242.0   32355.0

In [13]:
#plot monthly sales with sales difference
plot_data = [
    go.Scatter(
        x=df_diff['date'],
        y=df_diff['diff'],
    )
]
plot_layout = go.Layout(
        title='Montly Sales'
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
pyoff.iplot(fig)

In [14]:
#We need to use previous monthly sales data to forecast the next ones
#create dataframe for transformation from time series to supervised
df_supervised = df_diff.drop(['prev_sales'],axis=1)
#adding lags
for inc in range(1,13):
    field_name = 'lag_' + str(inc)
    df_supervised[field_name] = df_supervised['diff'].shift(inc)
#drop null values
df_supervised = df_supervised.dropna().reset_index(drop=True)

In [15]:
df_supervised.head()

date   sales      diff     lag_1     lag_2     lag_3     lag_4  \
0 2014-02-01  529117    3130.0   19380.0 -186036.0   36056.0  -33320.0   
1 2014-03-01  704301  175184.0    3130.0   19380.0 -186036.0   36056.0   
2 2014-04-01  788914   84613.0  175184.0    3130.0   19380.0 -186036.0   
3 2014-05-01  882877   93963.0   84613.0  175184.0    3130.0   19380.0   
4 2014-06-01  906842   23965.0   93963.0   84613.0  175184.0    3130.0   

      lag_5     lag_6    lag_7    lag_8    lag_9   lag_10    lag_11    lag_12  
0  -76854.0  -89161.0  60325.0  32355.0  80968.0  64892.0  157965.0    4513.0  
1  -33320.0  -76854.0 -89161.0  60325.0  32355.0  80968.0   64892.0  157965.0  
2   36056.0  -33320.0 -76854.0 -89161.0  60325.0  32355.0   80968.0   64892.0  
3 -186036.0   36056.0 -33320.0 -76854.0 -89161.0  60325.0   32355.0   80968.0  
4   19380.0 -186036.0  36056.0 -33320.0 -76854.0 -89161.0   60325.0   32355.0

In [16]:
# Import statsmodels.formula.api
import statsmodels.formula.api as smf
# Define the regression formula
model = smf.ols(formula='diff ~ lag_1', data=df_supervised)
# Fit the regression
model_fit = model.fit()
# Extract the adjusted r-squared
regression_adj_rsq = model_fit.rsquared_adj
print(regression_adj_rsq)

0.02893426930900389


In [17]:
# Import statsmodels.formula.api
import statsmodels.formula.api as smf
# Define the regression formula
model = smf.ols(formula='diff ~ lag_1 + lag_2 + lag_3 + lag_4 + lag_5', data=df_supervised)
# Fit the regression
model_fit = model.fit()
# Extract the adjusted r-squared
regression_adj_rsq = model_fit.rsquared_adj
print(regression_adj_rsq)

0.4406493613886946


In [18]:
# Import statsmodels.formula.api
import statsmodels.formula.api as smf
# Define the regression formula
model = smf.ols(formula='diff ~ lag_1 + lag_2 + lag_3 + lag_4 + lag_5 + lag_6 + lag_7 + lag_8 + lag_9 + lag_10 + lag_11 + lag_12', data=df_supervised)
# Fit the regression
model_fit = model.fit()
# Extract the adjusted r-squared
regression_adj_rsq = model_fit.rsquared_adj
print(regression_adj_rsq)

0.9795722233296558


In [19]:
#import MinMaxScaler and create a new dataframe for LSTM model
from sklearn.preprocessing import MinMaxScaler
df_model = df_supervised.drop(['sales','date'],axis=1)
#split train and test set
train_set, test_set = df_model[0:-6].values, df_model[-6:].values

In [20]:
train_set

array([[   3130.,   19380., -186036.,   36056.,  -33320.,  -76854.,
         -89161.,   60325.,   32355.,   80968.,   64892.,  157965.,
           4513.],
       [ 175184.,    3130.,   19380., -186036.,   36056.,  -33320.,
         -76854.,  -89161.,   60325.,   32355.,   80968.,   64892.,
         157965.],
       [  84613.,  175184.,    3130.,   19380., -186036.,   36056.,
         -33320.,  -76854.,  -89161.,   60325.,   32355.,   80968.,
          64892.],
       [  93963.,   84613.,  175184.,    3130.,   19380., -186036.,
          36056.,  -33320.,  -76854.,  -89161.,   60325.,   32355.,
          80968.],
       [  23965.,   93963.,   84613.,  175184.,    3130.,   19380.,
        -186036.,   36056.,  -33320.,  -76854.,  -89161.,   60325.,
          32355.],
       [  82168.,   23965.,   93963.,   84613.,  175184.,    3130.,
          19380., -186036.,   36056.,  -33320.,  -76854.,  -89161.,
          60325.],
       [-103414.,   82168.,   23965.,   93963.,   84613.,  175184.,
  

In [21]:
#apply Min Max Scaler
scaler = MinMaxScaler(feature_range=(-1, 1))
scaler = scaler.fit(train_set)
# reshape training set
train_set = train_set.reshape(train_set.shape[0], train_set.shape[1])
train_set_scaled = scaler.transform(train_set)
# reshape test set
test_set = test_set.reshape(test_set.shape[0], test_set.shape[1])
test_set_scaled = scaler.transform(test_set)

In [22]:
train_set_scaled

array([[ 0.07686073,  0.15255919, -0.80434393,  0.23024212, -0.0447346 ,
        -0.25830878, -0.3186859 ,  0.40696724,  0.26794062,  0.50957454,
         0.42966779,  0.8922929 ,  0.12955024],
       [ 0.8783514 ,  0.07686073,  0.15255919, -0.80434393,  0.29561828,
        -0.0447346 , -0.25830878, -0.33606217,  0.40696724,  0.26794062,
         0.50957454,  0.42966779,  0.8922929 ],
       [ 0.45643845,  0.8783514 ,  0.07686073,  0.15255919, -0.79394659,
         0.29561828, -0.0447346 , -0.27488947, -0.33606217,  0.40696724,
         0.26794062,  0.50957454,  0.42966779],
       [ 0.49999418,  0.45643845,  0.8783514 ,  0.07686073,  0.21380721,
        -0.79394659,  0.29561828, -0.05850103, -0.27488947, -0.33606217,
         0.40696724,  0.26794062,  0.50957454],
       [ 0.1739178 ,  0.49999418,  0.45643845,  0.8783514 ,  0.13408606,
         0.21380721, -0.79394659,  0.28633667, -0.05850103, -0.27488947,
        -0.33606217,  0.40696724,  0.26794062],
       [ 0.44504874,  0.173917

In [23]:
X_train, y_train = train_set_scaled[:, 1:], train_set_scaled[:, 0:1]
X_train = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])
X_test, y_test = test_set_scaled[:, 1:], test_set_scaled[:, 0:1]
X_test = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])

In [24]:
import keras
from keras.layers import Dense
from keras.models import Sequential
from keras.optimizers import Adam 
from keras.callbacks import EarlyStopping
from keras.utils import np_utils
from keras.layers import LSTM
from sklearn.model_selection import KFold, cross_val_score, train_test_split

In [25]:
model = Sequential()
model.add(LSTM(4, batch_input_shape=(1, X_train.shape[1], X_train.shape[2]), stateful=True))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(X_train, y_train, epochs=100, batch_size=1, verbose=1, shuffle=False)

Epoch 1/100
41/41 [==============================] - 1s 1ms/step - loss: 0.2926
Epoch 2/100
41/41 [==============================] - 0s 2ms/step - loss: 0.2448
Epoch 3/100
41/41 [==============================] - 0s 954us/step - loss: 0.2033
Epoch 4/100
41/41 [==============================] - 0s 1ms/step - loss: 0.1700
Epoch 5/100
41/41 [==============================] - 0s 984us/step - loss: 0.1435
Epoch 6/100
41/41 [==============================] - 0s 983us/step - loss: 0.1227
Epoch 7/100
41/41 [==============================] - 0s 1ms/step - loss: 0.1063
Epoch 8/100
41/41 [==============================] - 0s 1ms/step - loss: 0.0932
Epoch 9/100
41/41 [==============================] - 0s 1ms/step - loss: 0.0825
Epoch 10/100
41/41 [==============================] - 0s 1ms/step - loss: 0.0733
Epoch 11/100
41/41 [==============================] - 0s 1ms/step - loss: 0.0651
Epoch 12/100
41/41 [==============================] - 0s 1ms/step - loss: 0.0577
Epoch 13/100
41/41 [===========

In [26]:
y_pred = model.predict(X_test,batch_size=1)

6/6 [==============================] - 0s 1ms/step


In [27]:
#reshape y_pred
y_pred = y_pred.reshape(y_pred.shape[0], 1, y_pred.shape[1])
#rebuild test set for inverse transform
pred_test_set = []
for index in range(0,len(y_pred)):
    print(np.concatenate([y_pred[index],X_test[index]],axis=1))
    pred_test_set.append(np.concatenate([y_pred[index],X_test[index]],axis=1))
#reshape pred_test_set
pred_test_set = np.array(pred_test_set)
pred_test_set = pred_test_set.reshape(pred_test_set.shape[0], pred_test_set.shape[2])
#inverse transform
pred_test_set_inverted = scaler.inverse_transform(pred_test_set)

[[ 0.66131735  0.26695937  0.44344626  0.60355899  1.10628178  0.13866328
  -0.10745675 -1.02635392  0.24535439 -0.05787474 -0.31370458 -0.67437352
   0.68397168]]
[[-0.60748667  0.55964922  0.26695937  0.44344626  0.68877355  1.10628178
   0.13866328 -0.12204966 -1.02635392  0.24535439 -0.05787474 -0.31370458
  -0.67437352]]
[[-0.41733187 -0.61313659  0.55964922  0.26695937  0.52015228  0.68877355
   1.10628178  0.12731349 -0.12204966 -1.02635392  0.24535439 -0.05787474
  -0.31370458]]
[[-0.03924305 -0.36228353 -0.61313659  0.55964922  0.33428672  0.52015228
   0.68877355  1.10768225  0.12731349 -0.12204966 -1.02635392  0.24535439
  -0.05787474]]
[[ 0.18633822 -0.14316792 -0.36228353 -0.61313659  0.64253037  0.33428672
   0.52015228  0.68467253  1.10768225  0.12731349 -0.12204966 -1.02635392
   0.24535439]]
[[-1.02041745  0.23779333 -0.14316792 -0.36228353 -0.59257833  0.64253037
   0.33428672  0.51382935  0.68467253  1.10768225  0.12731349 -0.12204966
  -1.02635392]]


In [28]:
#create dataframe that shows the predicted sales
result_list = []
sales_dates = list(df[-7:].date)
act_sales = list(df[-7:].sales)
for index in range(0,len(pred_test_set_inverted)):
    result_dict = {}
    result_dict['pred_value'] = int(pred_test_set_inverted[index][0] + act_sales[index])
    result_dict['date'] = sales_dates[index+1]
    result_list.append(result_dict)
df_result = pd.DataFrame(result_list)

In [29]:
df_result

pred_value       date
0     1193217 2017-07-01
1     1027615 2017-08-01
2      923445 2017-09-01
3      913469 2017-10-01
4      917791 2017-11-01
5      696417 2017-12-01

In [30]:
df_sales_pred = pd.merge(df,df_result,on='date',how='left')
#plot actual and predicted
plot_data = [
    go.Scatter(
        x=df_sales_pred['date'],
        y=df_sales_pred['sales'],
        name='actual'
    ),
        go.Scatter(
        x=df_sales_pred['date'],
        y=df_sales_pred['pred_value'],
        name='predicted'
    )
    
]
plot_layout = go.Layout(
        title='Sales Prediction'
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
pyoff.iplot(fig)